In [1]:
import os
from lxml import html
import requests
from BeautifulSoup import BeautifulSoup
import json
import pandas as pd

In [2]:
schedule_data_file = 'schedule_data.json'
vessel_data_file = 'vessel_data.json'

In [5]:
# Pull existing data
page = requests.get('https://www.nwseaportalliance.com/operations/vessels')
soup = BeautifulSoup(page.text)

# Fetch the schedule data from the main page
schedule_db = {}

# Get the vessel list and href for vessel details
vessels = soup.findAll('td', {'class':'views-field views-field-title-1'})

# ETA, ETD, and and Terminal Data
eta = soup.findAll('div',{'class':'field field-name-field-vessel-eta field-type-datestamp field-label-hidden'})
etd = soup.findAll('div', {'class':'field field-name-field-vessel-etd field-type-datestamp field-label-hidden'})
harbor = soup.findAll('div', {'class':'field field-name-field-vessel-terminal field-type-text field-label-hidden'})

vessel_list = []
href_list = []
for i in xrange(len(vessels)):
    vessel = vessels[i]
    schedule_db[i] = {'vessel': vessel.findAll('a',href=True)[0].contents[0].strip(),
                     'href': vessel.findAll('a',href=True)[0]['href'],
                     'eta': eta[i].findAll('span')[0].contents[0].strip(),
                     'etd': etd[i].findAll('span')[0].contents[0].strip(),
                     'harbor': harbor[i].contents[0].split('\n')[-1].strip()}

In [6]:
# Look up ship data

# Try to load an existing vessel dict
if os.path.isfile(vessel_data_file):
    try:
        with open(vessel_data_file, 'r') as fp:
            vessel_dict = json.load(fp)
    except:
        vessel_dict = {}
else:
    vessel_dict = {}

# If ship doesn't already exist in our database, collect it
for entry_id, entry in schedule_db.iteritems():
    vessel = schedule_db[entry_id]['vessel'] 

    if vessel not in vessel_dict.keys():
        print 'adding: ' + vessel
        vessel_dict[vessel] = {}
        page = requests.get('https://www.nwseaportalliance.com' + schedule_db[entry_id]['href'])
        soup = BeautifulSoup(page.text)

        # # Get the list of class names to search
        vessel_class_names = []
        for field in [i['class'] for i in soup.findAll('div')[1:]]:
            if 'field' in str(field):
                vessel_class_names.append(field)

        for class_name in vessel_class_names:
            field_name = class_name.split(' ')[1].split('-')[-1]
            field_val = soup.findAll('div', {'class':class_name})[0].contents[0].split('\n')[-1].strip()
            vessel_dict[vessel][field_name] = field_val
            
with open(vessel_data_file, 'w') as fp:
    json.dump(vessel_dict, fp)

# Analysis

In [59]:
def dict_to_df(_dict):
    df = pd.DataFrame().from_dict(_dict).T
    df.reset_index(inplace=True)
    
    return df

In [67]:
# Load vessel data as df
vessel_df = dict_to_df(vessel_dict)
vessel_df['vessel'] = vessel_df['index']
# drop the old index and empty eta, etd columns
vessel_df.drop(['index','eta','etd'], axis=1, inplace=True)
vessel_df.head()

,agent,berth,cargotype,draught,harbor,length,stevedore,terminal,width,vessel
0,HAPAG-LLOYD (AMERICA) INC,B,CONTAINERS,50.85,TAC,1145.01,WASHINGTON UNITED TERMINALS,Washington United Terminals,149.61,APL GWANGYANG
1,K LINE AMERICA INC,A,AUTOS,31.07,TAC,590.52,JONES STEVEDORING COMPANY,Blair Terminal,105.64,ARCADIA HIGHWAY
2,WESTWOOD SHIPPING LINES,3,CONTAINERS / GC,38.06,TAC,685.37,PORTS AMERICA GROUP/PCT,Terminal 3,97.93,BARDU
3,KIRBY OFFSHORE MARINE,A,LAY BERTH,0.00,TAC,347.77,,Weyco Log,0.00,BARGE KIRBY OFFSHORE MARINE
4,OLYMPIC TUG &amp; BARGE,A,LAY BERTH,0.00,TAC,347.77,,Blair Terminal,0.00,BARGE OLYMPIC


In [68]:
# Load the schedule data as df
schedule_df = dict_to_df(schedule_db)
schedule_df.drop('index',axis=1,inplace=True)
schedule_df.head()

,eta,etd,harbor,href,vessel
0,12/05/2015 - 8:00am,09/10/2016 - 6:00pm,APM Terminals,/node/340857,MATSON PRODUCER
1,03/29/2016 - 8:00am,11/15/2016 - 8:00am,TOTE,/node/340896,SS EL YUNQUE
2,07/19/2016 - 7:00pm,07/30/2016 - 6:00pm,Weyco Log,/node/340914,BARGE SAUSE BROTHERS
3,07/25/2016 - 2:00pm,07/28/2016 - 2:00pm,Weyco Log,/node/340915,BARGE KIRBY OFFSHORE MARINE
4,07/26/2016 - 3:00pm,07/28/2016 - 3:00pm,Blair Terminal,/node/340860,BARGE OLYMPIC


In [69]:
# Join ship data to the schedule 
pd.merge(schedule_df, vessel_df,on='vessel')

,eta,etd,harbor_x,href,vessel,agent,berth,cargotype,draught,harbor_y,length,stevedore,terminal,width
0,12/05/2015 - 8:00am,09/10/2016 - 6:00pm,APM Terminals,/node/340857,MATSON PRODUCER,MATSON,B,LAY BERTH,34.09,TAC,720.54,,APM Terminals,95.18
1,03/29/2016 - 8:00am,11/15/2016 - 8:00am,TOTE,/node/340896,SS EL YUNQUE,TOTE MARITIME ALASKA INC,B,LAY BERTH,29.60,TAC,790.78,,TOTE,92.03
2,07/19/2016 - 7:00pm,07/30/2016 - 6:00pm,Weyco Log,/node/340914,BARGE SAUSE BROTHERS,SAUSE BROS. OCEAN TOWING CO.,B,LAY BERTH,0.00,TAC,347.77,,Weyco Log,0.00
3,07/25/2016 - 2:00pm,07/28/2016 - 2:00pm,Weyco Log,/node/340915,BARGE KIRBY OFFSHORE MARINE,KIRBY OFFSHORE MARINE,A,LAY BERTH,0.00,TAC,347.77,,Weyco Log,0.00
4,08/04/2016 - 8:00pm,08/05/2016 - 7:00pm,East Blair 1,/node/340869,BARGE KIRBY OFFSHORE MARINE,KIRBY OFFSHORE MARINE,A,LAY BERTH,0.00,TAC,347.77,,Weyco Log,0.00
5,07/26/2016 - 3:00pm,07/28/2016 - 3:00pm,Blair Terminal,/node/340860,BARGE OLYMPIC,OLYMPIC TUG &amp; BARGE,A,LAY BERTH,0.00,TAC,347.77,,Blair Terminal,0.00
6,07/29/2016 - 6:00pm,07/31/2016 - 6:00am,Terminal 7 A/B,/node/340903,BARGE OLYMPIC,OLYMPIC TUG &amp; BARGE,A,LAY BERTH,0.00,TAC,347.77,,Blair Terminal,0.00
7,08/05/2016 - 8:00pm,08/06/2016 - 7:00am,Blair Terminal,/node/340861,BARGE OLYMPIC,OLYMPIC TUG &amp; BARGE,A,LAY BERTH,0.00,TAC,347.77,,Blair Terminal,0.00
8,08/02/2016 - 6:00am,08/04/2016 - 3:00am,Washington United Terminals,/node/340908,HYUNDAI COURAGE,WASHINGTON UNITED TERMINALS,B,CONTAINERS,47.67,TAC,1114.24,WASHINGTON UNITED TERMINALS,Washington United Terminals,149.74
9,08/03/2016 - 3:00pm,08/05/2016 - 5:00am,Pierce County Terminal,/node/340880,EVER SHINE,EVERGREEN SHIPPING AGENCY,B,CONTAINERS,46.59,TAC,984.22,PORTS AMERICA GROUP/PCT,Pierce County Terminal,140.78
